In [1]:
import nltk
from nltk import Text
import random
import numpy as np
from datetime import datetime as dt
import os
import glob
from tqdm import tqdm
cwd = os.getcwd()

### Example Process

##### Setup

##### Feature Extractor

##### Training

### Real Data

In [2]:
%%time
texts = []
files_90s = glob.glob(cwd+"\\90_congress\\*.txt")
for i,file in enumerate(tqdm(files_90s)):
    try:
        file_sub = open(file, "r")
        text = file_sub.read()
        file_sub.close()
        
    except UnicodeDecodeError:
        continue
    tokens = nltk.word_tokenize(text)
    tokens = tokens[int(len(tokens)*.1):int(len(tokens)*.9)]
    j = 0
    while j < len(tokens):
        if(not(tokens[j].lower().isalpha())):
            del(tokens[j])
            j-=1
        j+=1
    texts.append((tokens,"90s"))
files_10s = glob.glob(cwd+"\\10_congress\\*.txt")
for i,file in enumerate(tqdm(files_10s)):
    try:
        file_sub = open(file, "r")
        text = file_sub.read()
        file_sub.close()
    except UnicodeDecodeError:
        continue
    tokens = nltk.word_tokenize(text)
    tokens = tokens[int(len(tokens)*.1):int(len(tokens)*.9)]
    j = 0
    while j < len(tokens):
        if(not(tokens[j].lower().isalpha())):
            del(tokens[j])
            j-=1
        j+=1
    texts.append((tokens,"10s"))

    # if(i > 3):
    #     break
all_words = [x for xs in texts for x in xs[0]]
all_words = list(nltk.FreqDist(all_words))

100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [07:20<00:00, 11.28s/it]


CPU times: total: 3min 52s
Wall time: 8min 38s


In [3]:
print("\n=== Concordance Analysis with POS Tagging ===")

# Create Text objects for each decade
pos_1990s = []
pos_2010s = []

text_1990s = []
text_2010s = []

for i in tqdm(texts):
    text, decade = i
    pos_text = nltk.pos_tag(text)
    Text_temp = Text(text)
    if(decade == "90s"):
        pos_1990s.append(pos_text)
        text_1990s.append(Text_temp)
    elif(decade == "10s"):
        pos_2010s.append(pos_text)
        text_2010s.append(Text_temp)
# text_1990s_tokens = [word for tokens, decade in texts if decade == "90s" for word in tokens]
# text_2010s_tokens = [word for tokens, decade in texts if decade == "10s" for word in tokens]

# print("1")
# # POS tagging
# pos_1990s = nltk.pos_tag(text_1990s_tokens)
# pos_2010s = nltk.pos_tag(text_2010s_tokens)

# print("2")

# # Recreate Text objects with original tokens
# text_1990s = Text(text_1990s_tokens)
# text_2010s = Text(text_2010s_tokens)

# print("3")

# # List of words for concordance analysis

# print("5")
# Display concordances and POS tags for each word
# display_concordances_with_pos(word_list, text_1990s, text_2010s, pos_1990s, pos_2010s)


=== Concordance Analysis with POS Tagging ===


100%|██████████████████████████████████████████████████████████████████████████████████| 69/69 [02:44<00:00,  2.39s/it]


In [4]:
word_list = ["Miss","Internet","Brazilian","Abu","Johnston","Mrs"]

def display_concordances_with_pos(word_list, text_1990s, text_2010s, pos_1990s, pos_2010s, width=80, lines=10):
    for word in tqdm(word_list):
        print(f"\n=== Concordance and POS for '{word}' ===")
        
        # POS in 1990s text
        print("\n--- In 1990s Text ---")
        try:
            for text,pos_temp in zip(text_1990s,pos_1990s):
                if word in text:
                    text.concordance(word, width=width, lines=lines)
                tags_1990s = [pos for token, pos in pos_temp if token.lower() == word.lower()]
                if(len(tags_1990s) > 0):
                    print(f"POS Tags in 1990s: {tags_1990s[:10]}")
        except ValueError:
            print(f"'{word}' not found in 1990s text.")
        
        # POS in 2010s text
        print("\n--- In 2010s Text ---")
        try:
            for text,pos_temp in zip(text_2010s,pos_2010s):
                if word in text:
                    text.concordance(word, width=width, lines=lines)
                tags_2010s = [pos for token, pos in pos_temp if token.lower() == word.lower()]
                if(len(tags_2010s) > 0):
                    print(f"POS Tags in 2010s: {tags_2010s[:10]}")
        except ValueError:
            print(f"'{word}' not found in 2010s text.")

In [5]:
display_concordances_with_pos(word_list,text_1990s,text_2010s,pos_1990s,pos_2010s)

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]


=== Concordance and POS for 'Miss' ===

--- In 1990s Text ---
POS Tags in 1990s: ['VB']
POS Tags in 1990s: ['NN']
Displaying 3 of 3 matches:
ent international beauty contests the Miss Universe and the Miss World competiti
ty contests the Miss Universe and the Miss World competitions Thousands of young
ps protested in New Delhi against the Miss India contest Future Trends Bv the ce
POS Tags in 1990s: ['NNP', 'NNP', 'NNP']
POS Tags in 1990s: ['VBD', 'JJ', 'NNP']
POS Tags in 1990s: ['NN', 'VB']

--- In 2010s Text ---
Displaying 10 of 16 matches:
ve Vice President Mrs Stowe Treasurer Miss Perot cretary Miss J Eve Minute Secre
rs Stowe Treasurer Miss Perot cretary Miss J Eve Minute Secretary Harvey Paget B
tor Patronesses Mrs T Butterfield and Miss Butterfield President Wm Gilbert Vice
and Secretary Lough District Visitors Miss Godet Miss Stewart Miss Hunt Miss Ste
ry Lough District Visitors Miss Godet Miss Stewart Miss Hunt Miss Stewart Miss S
rict Visitors Miss Godet Miss Stewart Miss Hu

 17%|██████████████                                                                      | 1/6 [00:02<00:14,  2.97s/it]

Displaying 10 of 1013 matches:
 came back The score of the match was MISS HAZEL HOTCHKISS National Champion and
fell below the standard of a year ago Miss Hazel Hotchkiss the holder came on fr
r earlier and although her challenger Miss Louise mond put up a much stronger ga
onger game than had Barger Wallach in Miss liotchkiss play was decidedly more er
list were those of Barger Wallach and Miss Evelyn Sears champions of and respect
nt in past seasons In the third round Miss Lois Moyes of Toronto and Miss Green 
 round Miss Lois Moyes of Toronto and Miss Green of Philadelphia were eliminated
Green of Philadelphia were eliminated Miss Wildey was the vanquisher of Miss Moy
ted Miss Wildey was the vanquisher of Miss Moyes and although the winner was con
which promised great things in future Miss Moyes failed to score a set and was d
POS Tags in 2010s: ['NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP']
Displaying 10 of 1754 matches:
n defeated Alexander and Reals Wright 

 33%|████████████████████████████                                                        | 2/6 [00:04<00:09,  2.29s/it]

...

 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [00:09<00:01,  1.75s/it]

POS Tags in 2010s: ['NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP']
Displaying 1 of 1 matches:
d Company are the publishers of the Johnston maps They intend to submit other ma
POS Tags in 2010s: ['NNP']
Displaying 10 of 57 matches:
rdays and Sundays starting June Ham Johnston who won the premier title three tim
reenberg Knowlton Chapman D William Johnston Willis Davis by default Curley d St
n Batkin Knowlton Nourse by default Johnston Curley Strachan Knowlton FINAL John
ston Curley Strachan Knowlton FINAL Johnston Strachan WOMEN SINGLES ST Miss Doro
ingles match of two sets only Billy Johnston and M McLoughlin divided the honors
and M McLoughlin divided the honors Johnston won the first set at McLoi playing 
zzled over the net He seemingly had Johnston at his mercy in th and won easily a
se Hahn and Winne Cornett and Dixon Johnston and Sinsabaugh Rager and Busch Brow
 Donnell Warren and Hart by default Johnston and Sinsaba Hahn and Winne Browne a
dy Barker and Don

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.58s/it]

POS Tags in 2010s: ['NNP']
Displaying 3 of 3 matches:
county made careful examination as to Mrs Murray qualification and he appointed
id think it good politics to nominate Mrs Murray and they put a man in Mrs Murr
nate Mrs Murray and they put a man in Mrs Murray was nominated by the Prohibiti
POS Tags in 2010s: ['NNP', 'NNP', 'NNP']
Displaying 10 of 33 matches:
i default Miss Hofer Miss Bell Ramsey Mrs Hofer default Miss A Levy Miss Hanna 
 Miss Grove THIRD rstedt Waring Rogge Mrs Pritchard Miss Goss Winn Miss Pollak 
iss Green and Miss Winn Pritchard and Mrs Eberhart Grove and Bernhard Stenz Wil
ss Goss Miss Pollak and Miss Bernhard Mrs Green and s Winn Humphries and Miss H
fault Miss Ethel Tyndale Arthur Deane Mrs Frederic Damrau Huff by default Miss 
allin Cassebeer Hartwell Miss Tyndale Mrs Damrau Miss Gilleaudeau LeRoy Manheim
of Los Angeles Miss Evelyn Seavey and Mrs Peer of Kansas City and Thirlby of Tr
t and Miss Timm Miss Waldo and Miller Mrs Peer and Thirlby Miss Steever and 

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.86s/it]


In [6]:
len(all_words)

147667

In [7]:
word_features = list(all_words)[:4000]

def text_features(text): # define feature extractor
    document_words = set(text) # builds an iterable from document (likely just a list/set of words from the movie_review)
                                    # sets are very quick to check (word in document_words) compared to checking lists
    features = {} # dictionary
    for word in word_features: # Iterate through all 2000 most common words in the corpus
        features['contains({})'.format(word)] = (word in document_words) # create dictionary index with value TRUE or FALSE
                                                                        # Depending on if word is in document_words
    return features

In [8]:
print(list(text_features(texts[0][0]).items())[:10]) # test

[('contains(the)', True), ('contains(of)', True), ('contains(and)', True), ('contains(in)', True), ('contains(to)', True), ('contains(a)', True), ('contains(The)', True), ('contains(for)', True), ('contains(by)', True), ('contains(was)', True)]


### Training

In [9]:
featuresets = [(text_features(t),c) for (t,c) in texts]

In [10]:
train_set,test_set = featuresets[:int(len(featuresets)*0.8)],featuresets[int(len(featuresets)*0.8):] 
# 80 20 split for test and train

In [11]:
classifier = nltk.NaiveBayesClassifier.train(train_set) # train using built in

In [12]:
print(nltk.classify.accuracy(classifier, test_set)) # see the accuracy of the program

1.0


In [13]:
classifier.show_most_informative_features(-1) # 5 most influential features

Most Informative Features
          contains(Miss) = True              10s : 90s    =     20.4 : 1.0
      contains(Internet) = True              10s : 90s    =     14.1 : 1.0
      contains(Johnston) = True              10s : 90s    =     14.1 : 1.0
           contains(Mrs) = True              10s : 90s    =     14.1 : 1.0
           contains(Sec) = True              10s : 90s    =     14.1 : 1.0
     contains(paragraph) = True              10s : 90s    =     14.1 : 1.0
        contains(shades) = True              10s : 90s    =     14.1 : 1.0
      contains(Atkinson) = True              10s : 90s    =     11.0 : 1.0
       contains(COMPANY) = True              10s : 90s    =     11.0 : 1.0
         contains(Cloth) = True              10s : 90s    =     11.0 : 1.0
         contains(EXTRA) = True              10s : 90s    =     11.0 : 1.0
            contains(IK) = True              10s : 90s    =     11.0 : 1.0
        contains(Patent) = True              10s : 90s    =     11.0 : 1.0

In [14]:
file90 = cwd+"\\domestictrendsto00osbo\\domestictrendsto00osbo_djvu.txt"
file90_sub = open(file90, "r",encoding="utf8")
text90 = file90_sub.read()
file90_sub.close()
file10 = cwd+"\\pribilofislandsa00lind\\pribilofislandsa00lind_djvu.txt"
file10_sub = open(file10, "r",encoding="utf8")
text10 = file10_sub.read()
file10_sub.close()

In [15]:
text90 = nltk.word_tokenize(text90)
feature90 = text_features(text90)
text10 = nltk.word_tokenize(text10)
feature10 = text_features(text10)

In [16]:
classifier.classify_many([feature90,feature10])

['90s', '10s']

### Decision Tree

In [17]:
classifier2 = nltk.DecisionTreeClassifier.train(train_set)

In [18]:
print(nltk.classify.accuracy(classifier2, test_set)) # see the accuracy of the program

1.0


In [19]:
print(classifier2)

contains(Miss)=False? ................................. 10s
  contains(online)=False? ............................. 90s
  contains(online)=True? .............................. 10s
    contains(notice)=False? ........................... 90s
    contains(notice)=True? ............................ 10s
contains(Miss)=True? .................................. 10s
  contains(clan)=False? ............................... 10s
  contains(clan)=True? ................................ 90s



In [20]:
classifier2.classify_many([feature90,feature10])

['90s', '10s']